# **Ultimate ComfyUI Playground**
### Your All-in-One AI Image & Video Creation Studio
---Welcome to your advanced ComfyUI environment...

## 1. Setup API Keys & Mount Google Drive

In [ ]:
# This cell will mount your Google Drive...
from google.colab import drive, userdata
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Error: {e}")

## 2. Environment Setup

In [ ]:
#@title ## Run Setup
import os
PERFORM_INITIAL_SETUP = True #@param {type:"boolean"}
# ... rest of setup code ...

## 3. Model & Workflow Downloader

In [ ]:
#@title ## Download Models & Workflows
import os
# ... rest of download code ...

## 4. Launch ComfyUI

In [ ]:
#@title ## Start ComfyUI Server
import os, threading
# ... rest of launch code ...

---# **Advanced Usage: In-Notebook Workflows**

### 5.1 Text-to-Image Workflow

In [ ]:
#@title Generate Image from Prompt
import websocket
import uuid
import json
import urllib.request
import urllib.parse
from PIL import Image
import io
from IPython.display import display

# --- Parameters ---
SERVER_ADDRESS = "127.0.0.1:8188"
PROMPT = "a beautiful photograph of a majestic cat, cinematic lighting" #@param {type:"string"}
NEGATIVE_PROMPT = "ugly, distorted, deformed, blurry, text, watermark" #@param {type:"string"}
MODEL = "v1-5-pruned-emaonly.safetensors" #@param {type:"string"}
STEPS = 25 #@param {type:"integer"}
CFG = 7.5 #@param {type:"number"}
WIDTH = 512 #@param {type:"integer"}
HEIGHT = 512 #@param {type:"integer"}
SEED = 0 #@param {type:"integer"}

# --- Workflow Definition ---
t2i_workflow = {
  "3": {
    "class_type": "KSampler",
    "inputs": {
      "seed": SEED,
      "steps": STEPS,
      "cfg": CFG,
      "sampler_name": "euler",
      "scheduler": "normal",
      "denoise": 1,
      "model": ["4", 0],
      "positive": ["6", 0],
      "negative": ["7", 0],
      "latent_image": ["5", 0]
    }
  },
  "4": {
    "class_type": "CheckpointLoaderSimple",
    "inputs": {
      "ckpt_name": MODEL
    }
  },
  "5": {
    "class_type": "EmptyLatentImage",
    "inputs": {
      "width": WIDTH,
      "height": HEIGHT,
      "batch_size": 1
    }
  },
  "6": {
    "class_type": "CLIPTextEncode",
    "inputs": {
      "text": PROMPT,
      "clip": ["4", 1]
    }
  },
  "7": {
    "class_type": "CLIPTextEncode",
    "inputs": {
      "text": NEGATIVE_PROMPT,
      "clip": ["4", 1]
    }
  },
  "8": {
    "class_type": "VAEDecode",
    "inputs": {
      "samples": ["3", 0],
      "vae": ["4", 2]
    }
  },
  "9": {
    "class_type": "PreviewImage",
    "inputs": {
      "images": ["8", 0]
    }
  }
}

def queue_prompt(server_address, prompt, client_id):
    p = {"prompt": prompt, "client_id": client_id}
    data = json.dumps(p).encode('utf-8')
    req = urllib.request.Request(f"http://{server_address}/prompt", data=data)
    return json.loads(urllib.request.urlopen(req).read())

def get_image(server_address, filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen(f"http://{server_address}/view?{url_values}") as response:
        return response.read()

def get_history(server_address, prompt_id):
    with urllib.request.urlopen(f"http://{server_address}/history/{prompt_id}") as response:
        return json.loads(response.read())

def get_images(ws, server_address, prompt, client_id):
    prompt_id = queue_prompt(server_address, prompt, client_id)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(server_address, prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(server_address, image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images

client_id = str(uuid.uuid4())
ws = websocket.WebSocket()
ws.connect(f"ws://{SERVER_ADDRESS}/ws?clientId={client_id}")

print("Queueing prompt and waiting for result...")
images = get_images(ws, SERVER_ADDRESS, t2i_workflow, client_id)
ws.close()

# --- Display Images ---
for node_id in images:
    for image_data in images[node_id]:
        image = Image.open(io.BytesIO(image_data))
        print(f"Displaying image from node {node_id}")
        display(image)

print("Workflow complete!")